# Use Langchain to interact with a SQL database 

The following code showcases an example of the Databricks SQL Agent. With the Databricks SQL agent any Databricks users can interact with a specified schema in Databrick Unity Catalog and generate insights on their data.

## Requirements

- To use this notebook, please provide your OpenAI API Token.
- Databricks Runtime 13.3 ML and above

**SOURCE: https://docs.databricks.com/en/large-language-models/langchain.html**  

The example mentioned in the notebook has outdated packages. 

### Imports

Databricks recommends the latest version of `langchain` and the `databricks-sql-connector`.

In [ ]:
%pip install --upgrade openai

Note: you may need to restart the kernel using %restart_python or dbutils.library.restartPython() to use updated packages.
Note: you may need to restart the kernel using %restart_python or dbutils.library.restartPython() to use updated packages.


In [ ]:
%pip install --upgrade sqlalchemy

Note: you may need to restart the kernel using %restart_python or dbutils.library.restartPython() to use updated packages.
Note: you may need to restart the kernel using %restart_python or dbutils.library.restartPython() to use updated packages.


In [ ]:
%pip install --upgrade langchain databricks-sql-connector langchain-openai


Note: you may need to restart the kernel using %restart_python or dbutils.library.restartPython() to use updated packages.
Note: you may need to restart the kernel using %restart_python or dbutils.library.restartPython() to use updated packages.


In [ ]:
!pip show openai

Name: openai
Version: 1.43.0
Summary: The official Python library for the openai API
Home-page: 
Author: 
Author-email: OpenAI <support@openai.com>
License: 
Location: /local_disk0/.ephemeral_nfs/envs/pythonEnv-b86171d0-fead-464a-b1a9-0c410fa0016c/lib/python3.11/site-packages
Requires: anyio, distro, httpx, jiter, pydantic, sniffio, tqdm, typing-extensions
Required-by: langchain-openai


In [ ]:
!pip show langchain

Name: langchain
Version: 0.2.16
Summary: Building applications with LLMs through composability
Home-page: https://github.com/langchain-ai/langchain
Author: 
Author-email: 
License: MIT
Location: /local_disk0/.ephemeral_nfs/envs/pythonEnv-b86171d0-fead-464a-b1a9-0c410fa0016c/lib/python3.11/site-packages
Requires: aiohttp, langchain-core, langchain-text-splitters, langsmith, numpy, pydantic, PyYAML, requests, SQLAlchemy, tenacity
Required-by: 


In [ ]:
!pip show databricks-sql-connector

Name: databricks-sql-connector
Version: 3.4.0
Summary: Databricks SQL Connector for Python
Home-page: 
Author: Databricks
Author-email: databricks-sql-connector-maintainers@databricks.com
License: Apache-2.0
Location: /local_disk0/.ephemeral_nfs/envs/pythonEnv-b86171d0-fead-464a-b1a9-0c410fa0016c/lib/python3.11/site-packages
Requires: lz4, numpy, oauthlib, openpyxl, pandas, pyarrow, requests, thrift, urllib3
Required-by: 


In [ ]:
import sys  
print("Python version:", sys.version)  

Python version: 3.11.0rc1 (main, Aug 12 2022, 10:02:14) [GCC 11.2.0]


In [ ]:
dbutils.library.restartPython()

In [ ]:
import os
os.environ["OPENAI_API_KEY"] = ""

### SQL Database Agent

This is an example of how to interact with a certain schema in Unity Catalog. Please note that the agent can't create new tables or delete tables. It can only query tables.

The database instance is created within:
```
db = SQLDatabase.from_databricks(catalog="...", schema="...")
```
And the agent (and the required tools) are created by:
```
toolkit = SQLDatabaseToolkit(db=db, llm=llm)
agent = create_sql_agent(llm=llm, toolkit=toolkit, **kwargs)
```

## Using Open AI

In [ ]:
from langchain.agents import create_sql_agent
from langchain.agents.agent_toolkits import SQLDatabaseToolkit
from langchain.sql_database import SQLDatabase
from langchain_openai import OpenAI

db = SQLDatabase.from_databricks(catalog="samples", schema="nyctaxi")
llm = OpenAI(temperature=.7)
toolkit = SQLDatabaseToolkit(db=db, llm=llm)
agent = create_sql_agent(llm=llm, toolkit=toolkit, verbose=True)

In [ ]:
agent.invoke("What is the longest trip distance and how long did it take?")



> Entering new SQL Agent Executor chain...
 To get the longest trip distance and duration, we need to query the trip table and join it with the trip details table to get the duration information.
Action: sql_db_query
Action Input: SELECT trip.trip_distance, trip_details.duration FROM trip INNER JOIN trip_details ON trip.trip_id = trip_details.trip_id ORDER BY trip.trip_distance DESC LIMIT 1Error: (databricks.sql.exc.ServerOperationError) [TABLE_OR_VIEW_NOT_FOUND] The table or view `samples`.`nyctaxi`.`trip` cannot be found. Verify the spelling and correctness of the schema and catalog.
If you did not qualify the name with a schema, verify the current_schema() output, or qualify the name with the correct schema and catalog.
To tolerate the error on drop use DROP VIEW IF EXISTS or DROP TABLE IF EXISTS. SQLSTATE: 42P01
[SQL: SELECT trip.trip_distance, trip_details.duration FROM trip INNER JOIN trip_details ON trip.trip_id = trip_details.trip_id ORDER BY trip.trip_distance DESC LIMIT 1]


{'input': 'What is the longest trip distance and how long did it take?',
 'output': 'Agent stopped due to iteration limit or time limit.'}

In [ ]:
agent.invoke("find out the zipcode of the trip with the highest fare")



> Entering new SQL Agent Executor chain...
 The zipcode information is most likely stored in the trips table, so we need to query that table and sort by fare in descending order to get the highest fare trip.
Action: sql_db_query
Action Input: SELECT zipcode FROM trips ORDER BY fare DESC LIMIT 1Error: (databricks.sql.exc.ServerOperationError) [UNRESOLVED_COLUMN.WITH_SUGGESTION] A column, variable, or function parameter with name `zipcode` cannot be resolved. Did you mean one of the following? [`pickup_zip`, `dropoff_zip`, `fare_amount`, `trip_distance`, `tpep_pickup_datetime`]. SQLSTATE: 42703; line 1 pos 7
[SQL: SELECT zipcode FROM trips ORDER BY fare DESC LIMIT 1]
(Background on this error at: https://sqlalche.me/e/20/4xp6) The error message indicates that the column 'zipcode' does not exist in the trips table. We need to use the sql_db_schema tool to get the correct column name.
Action: sql_db_schema
Action Input: trips
CREATE TABLE trips (
	tpep_pickup_datetime TIMESTAMP, 
	tpep_d

{'input': 'find out the zipcode of the trip with the highest fare',
 'output': 'The zipcode of the trip with the highest fare is 10013.'}

## Using Azure Open AI

In [ ]:
from langchain.agents import create_sql_agent
from langchain.agents.agent_toolkits import SQLDatabaseToolkit
from langchain.sql_database import SQLDatabase
from langchain_openai import OpenAI, AzureChatOpenAI

db = SQLDatabase.from_databricks(catalog="samples", schema="nyctaxi")

llm = AzureChatOpenAI(
    temperature=0.7,
    api_key="",
    api_version = "",
    azure_endpoint="",
    model="gpt4o"
)

toolkit = SQLDatabaseToolkit(db=db, llm=llm)
agent = create_sql_agent(llm=llm, toolkit=toolkit, verbose=True)

In [ ]:
agent.invoke("What is the longest trip distance and how long did it take?")



> Entering new SQL Agent Executor chain...
To answer this question, we need to know the tables in the database and understand their structure to identify where trip distance and trip duration data are stored. Let's start by listing all the tables in the database.

Action: sql_db_list_tables
Action Input: tripsThe database contains a table named "trips." To find the longest trip distance and its duration, we need to understand the schema of the "trips" table. This will help us identify the relevant columns for trip distance and trip duration.

Action: sql_db_schema
Action Input: trips
CREATE TABLE trips (
	tpep_pickup_datetime TIMESTAMP, 
	tpep_dropoff_datetime TIMESTAMP, 
	trip_distance FLOAT, 
	fare_amount FLOAT, 
	pickup_zip INT, 
	dropoff_zip INT
) USING DELTA
TBLPROPERTIES('delta.feature.allowColumnDefaults' = 'enabled')

/*
3 rows from trips table:
tpep_pickup_datetime	tpep_dropoff_datetime	trip_distance	fare_amount	pickup_zip	dropoff_zip
2016-02-14 16:52:13+00:00	2016-02-14 17:

{'input': 'What is the longest trip distance and how long did it take?',
 'output': 'Agent stopped due to iteration limit or time limit.'}

In [ ]:
agent.invoke("find out the zipcode of the trip with the highest fare")



> Entering new SQL Agent Executor chain...
To find the zipcode of the trip with the highest fare, I need to:

1. Identify which table contains information about trips and fares.
2. Check the schema of that table to understand the relevant columns.
3. Construct and execute a query to find the zipcode of the trip with the highest fare.

First, I will list all the tables in the database to identify the relevant table.

Action: sql_db_list_tables
Action Input: tripsThe database contains a table named `trips`. Now, I need to check the schema of the `trips` table to understand its columns and identify which columns contain information about the fare and zipcode.

Action: sql_db_schema
Action Input: trips
CREATE TABLE trips (
	tpep_pickup_datetime TIMESTAMP, 
	tpep_dropoff_datetime TIMESTAMP, 
	trip_distance FLOAT, 
	fare_amount FLOAT, 
	pickup_zip INT, 
	dropoff_zip INT
) USING DELTA
TBLPROPERTIES('delta.feature.allowColumnDefaults' = 'enabled')

/*
3 rows from trips table:
tpep_pickup_dat

{'input': 'find out the zipcode of the trip with the highest fare',
 'output': 'The zipcode of the trip with the highest fare has a pickup zipcode of 10013 and a dropoff zipcode of 7008.'}